In [2]:
import pandas as pd

In [3]:
lista_caminhos = ['C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/2_silver/One_Big_Table.parquet',
                  'C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/2_silver/One_Big_Table_02.parquet']

In [4]:
df_obt_vendas = pd.read_parquet(lista_caminhos[0])
df_obt_devolucao = pd.read_parquet(lista_caminhos[1])

### Criação das tabelas normalizadas
Iremos agora pegar a tabela OBT e criar as demais tabelas padronizadas e com algumas métricas.

In [6]:
# Dimensão Clientes
dim_cliente = df_obt_vendas[['id_cliente', 'primeiro_nome', 'sobrenome', 'email', 'genero', 
                  'data_nascimento', 'estado_civil', 'num_filhos', 'nivel_escolar']].drop_duplicates()

In [7]:
# Dimensão Produtos
dim_produto = df_obt_vendas[['sku', 'produto', 'marca', 'tipo_do_produto']].drop_duplicates()

In [8]:
# Dimensão Lojas
dim_loja = df_obt_vendas[['id_loja', 'nome_da_loja', 'quantidade_colaboradores', 'tipo', 
               'gerente_loja', 'documento_gerente', 'id_localidade','pais','continente']].drop_duplicates()

In [9]:
# Algumas métricas para adicionar na tabela fato

df_obt_vendas['receita'] = df_obt_vendas['preco_unitario'] * df_obt_vendas['qtd_vendida']
df_obt_vendas['custo'] = df_obt_vendas['custo_unitario'] * df_obt_vendas['qtd_vendida']
df_obt_vendas['lucro'] = df_obt_vendas['receita'] - df_obt_vendas['custo']

In [10]:
# Fato Vendas

fato_vendas = df_obt_vendas[['ordem_de_compra', 'data_da_venda', 'id_cliente', 'id_loja', 'sku',
                  'qtd_vendida', 'preco_unitario', 'custo_unitario', 'receita', 'lucro']]

In [11]:
# Dimensão Calendário

dim_tempo = df_obt_vendas[['data_da_venda']].drop_duplicates()

dim_tempo['ano'] = dim_tempo['data_da_venda'].dt.year
dim_tempo['mes'] = dim_tempo['data_da_venda'].dt.month
dim_tempo['dia'] = dim_tempo['data_da_venda'].dt.day
dim_tempo['dia_semana'] = dim_tempo['data_da_venda'].dt.day_name()
dim_tempo['trimestre'] = dim_tempo['data_da_venda'].dt.quarter

In [12]:
df_obt_devolucao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1809 entries, 0 to 1808
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   data_devolucao            1809 non-null   datetime64[ns]
 1   id_loja                   1809 non-null   int64         
 2   sku                       1809 non-null   object        
 3   qtd_devolvida             1809 non-null   int64         
 4   motivo_devolucao          1809 non-null   object        
 5   produto                   1809 non-null   object        
 6   marca                     1809 non-null   object        
 7   tipo_do_produto           1809 non-null   object        
 8   preco_unitario            1809 non-null   float64       
 9   custo_unitario            1809 non-null   float64       
 10  nome_da_loja              1809 non-null   object        
 11  quantidade_colaboradores  1809 non-null   int64         
 12  tipo                

In [13]:
# Criando coluna com o valor da devolução
df_obt_devolucao['valor_devolvido'] = df_obt_devolucao['preco_unitario'] * df_obt_devolucao['qtd_devolvida']


In [14]:
# Fato devolução

fato_devolucao = df_obt_devolucao[['data_devolucao', 'id_loja', 'sku', 'qtd_devolvida','preco_unitario', 'valor_devolvido','produto','marca','tipo_do_produto']]

### Resultado da normalização:

Conseguimos criar as 2 tabelas fato e as 3 tabelas dimensões. De bônus ainda entregamos uma tabela dim_calendario.

- fato_vendas
- fato_devolucao
- dim_produto
- dim_clientes
- dim_lojas
- dim_calendario

In [ ]:
# Exportando tabelas formatadas para a camada gold.

dim_cliente.to_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_clientes.parquet')
dim_loja.to_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_loja.parquet')
dim_produto.to_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_produtos.parquet')
dim_tempo.to_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_tempo.parquet')
fato_devolucao.to_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/fato_devolucao.parquet')
fato_vendas.to_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/fato_vendas.parquet')